## Projet de Recherche opérationnelle Delpeuch Sébastien & Decou Nathan
### Chargement des modules nécessaires pour le projet
Commencez par exécuter la cellule ci-dessous. Elle va charger tout le nécessaire pour faire fonctionner votre code. 
Si vous le lancez en ligne, __soyez patients__ (attendez que l'étoile à gauche devienne un nombre entre crochets).


In [1]:
include("jupy.jl")

   Updating registry at `~/.julia/registries/General`


   Updating git-repo `https://github.com/JuliaRegistries/General.git`


Fetching: [========================================>]  100.0 % %            ]  69.5 %

  Resolving package versions...
   Updating `~/.julia/environments/v1.4/Project.toml`
 [no changes]
   Updating `~/.julia/environments/v1.4/Manifest.toml`
 [no changes]
  Resolving package versions...
   Updating `~/.julia/environments/v1.4/Project.toml`
 [no changes]
   Updating `~/.julia/environments/v1.4/Manifest.toml`
 [no changes]


main (generic function with 1 method)

### Donnéees à utiliser

On rappelle les structures qui sont remplies par les méthodes de lecture de données.

#### data structure for data used to compute the balance cost
```
struct BalanceData
    resource1::Int64        # first resource
    resource2::Int64        # second resource
    target::Int64           # target aimed for the recource consumption
    weight::Int64           # weight of the balance cost
end
```

#### data struture for an instance of Google challenge problem
```
struct DataGoogle
    nbResources::Int64               # number of resources in the instance
    transientStatus::Array{Int64}    # table indicating if each resource is transient (O/1)
    weightLoadCost::Array{Int64}     # table indicating the weight of load cost for each resource

    nbMachines::Int64                # number of machines
    neighborhoods::Array{Int64}      # table indicating the neighborhood of each machine
    nbNeighborhoods::Int64           # total number of neighbordhoods
    locations::Array{Int64}          # table indicating the location of each machine
    nbLocations::Int64               # total number of locations
    softCapacities::Array{Int64,2}   # softcapacities[m,r] indicates the soft cap of machine m for resource r
    hardCapacities::Array{Int64,2}   # hardcapacities[m,r] indicates the hard cap of machine m for resource r
    machineMoveCosts::Array{Int64,2} # machineMoveCosts[m1,m2] indicates the cost for moving from m1 to m2

    nbServices::Int64                # number of services
    spreadMin::Array{Int64}          # for each service, its spread min
    dependences::Array{Array{Int64}} # for each service, a list of dependences (service numbers)

    nbProcess::Int64                 # number of processes
    servicesProcess::Array{Int64}    # for each process  its service
    processReq::Array{Int64,2}       # processReq[p,r] indicates the consumption of resource r by p
    processMoveCost::Array{Int64}    # cost of moving each process p

    nbBalanceCostData::Int64               # number of balance constraints
    balanceCostDataList::Array{BalanceData} # one for each balance target
    processMoveWeight::Int64        # weight for the process move cost
    serviceMoveWeight::Int64        # weight for the service move cost
    machineMoveWeight::Int64        # weight for the machine move cost

    initialAssignment # for each process, id of the machine in the initial solution
end
```

#### slution 
```
struct SolutionGoogle
    assignment::Array{Int64}
    cost::Int64
end
```





### Cellule à modifier
Insérez ci-dessous votre modèle dans la méthode **solveGoogle**. 

N'oubliez pas de faire un run sur la cellule quand vous avez fini. 

**Quelques informations sur la forme de notre rendu :** 

Pour résoudre le problème nous effectuons toujours la même démarche, en commentaire la contrainte / l'objectif que nous cherchons à résoudre puis 2-3 lignes d'explication théorique et finalement l'implémentation

Pour résoudre le problème nous avons utilisé beaucoup de fonction auxiliaires définit à l'intérieur de la fonction "solveGoogle", toutes les fonctions ont une courte description

Nous avons décidé de construire le fichier en suivant le sujet, toutes les contraintes et objectifs sont implémentés hormis C5.

In [38]:

function solveGoogle(data::DataGoogle, verbose::Bool)

    # Définition du modèle
    modele = Model(Cbc.Optimizer)
    set_optimizer_attribute(modele, "seconds", 120) # Pour que a1_5 ne soit pas infini normalement 
    # 2 minutes suffit à trouver une solution optimale
    set_optimizer_attribute(modele, "logLevel", 0) 
    
    ### Question 1 ###
    
    # C1 : Contrainte d'affectation 
    # L'idée est que chaque processus doit être affecté à une machine 
    # Nous définissons donc la variable x_{ij} qui vaut 1 si le processus i est affecté à la machine j
    # Mathématiquement cela se traduit par somme de j allant de 1 à M des x_{ij} égal 1 pour i allant de 1 à n
    @variable(modele,x[1:data.nbProcess,1:data.nbMachines]>=0, Bin) #Variable permettant de connaitre les affectations
    
    @constraint(modele, C1_affectation[p in 1:data.nbProcess], 
        sum(x[p,m] for m in 1:data.nbMachines) == 1)
    
    # C2 : Contrainte de capacité
    # La somme des demandes en ressources de tous les processus sur une machine doit être inférieur 
    # aux capacités de la machine sur chaque ressources
    # Mathématiquement : somme(p=1,P)  R(p,r) x_{p,m} < C(m,r) avec r de 1 à R et m de 1 à M
    
    @constraint(modele, C2_capacite[r in 1:data.nbResources, m in 1:data.nbMachines],
        sum(data.processReq[p, r] * x[p,m]  for p in 1:data.nbProcess) 
        <= data.hardCapacities[m, r])

   
    # C3 : Contrainte de dijonction 
    # L'idée de cette contrainte est d'empecher deux processus appartenant au même service de s'executer
    # sur la même machine 
    # Cela va se traduire par une vérification s'inspirant de l'exercice de coloration 
    # Ainsi, on impose que deux processus distincts (p1 et p2 par exemple) qui appartiennent au même
    # service implique que x(p1,m) + x(p2,m) <= 1 (par définition de x)
    
    @constraint(modele, C3_disjonction[m in 1:data.nbMachines, p1 in 1:data.nbProcess, p2 in 1:data.nbProcess;
        p1 != p2 #les deux processus doivent être distincts
        && data.servicesProcess[p1] == data.servicesProcess[p2]], #Test de l'égalité 
        x[p1,m] + x[p2,m] <= 1)

    # O3 : Objectif de coût de déplacement des processus
    # L'objectif est de minimiser la somme des couts des déplacements des processus 
    # En prevoyant la question "coefficient dans l'objectif" nous ne créons pas un "@objectif"
    # mais juste une variable le contenant
    # Mathématiquement cela se traduit par la somme (sur tous les processus) du cout de mouvement 
    # si et seulement si le processus à bougé (ce que l'on peut traduire par 
    # 1 - x(p,position initiale) qui vaut 0 si le processus n'a pas bougé)
    objectif3=sum(data.processMoveCost[p] * (1 - x[p,data.initialAssignment[p]]) 
        for p in 1:data.nbProcess)
    
    ### Fin Question 1 ###
    
    ### Question 2 ### 
    # Le but de cette question est d'ajouter des objectifs au modèle 
    # Plus particulièrement avoir à la fin 5 "couts" que l'on réunira dans un objectif en fonction 
    # de différents poids
    
    # O1 : Objectif de capacité de sécurité
    # On souhaite rester en dessous de la limite de sécurité pour chaque ressource sur chaque 
    # machine formellement nous allons définir le cout de dépassement comme 
    # la somme (sur toute les machines, sur toutes les ressources) 
    # w^{LC}(r)*max(0,U(m,r)-SC(m,r)) avec U le nombre d'unité de ressource que
    # consomment les processus sur une machine et SC la limite de sécurité sur cette ressource
    
     # Nous allons créer une fonction permettant d'obtenir pour une ressource r et une machine m
    # le nombre d'unité de ressource consommé sur cette machine 
    function ressourcesUtilisees(m,r)
       return sum(x[p,m]*data.processReq[p,r] for p in 1:data.nbProcess) 
    end
    
     # Nous créons maintenant une fonction permettant de calculer U(m,r)-SC(m,r)
    function diffUtiliseesSoft(m,r)
       return ressourcesUtilisees(m,r)-Int64(data.softCapacities[m,r])
    end
    
    # Variable et contrainte permettant de faire le max(0,U-SC)
    @variable(modele, maxSC[1:data.nbMachines, 1:data.nbResources] >= 0, Int)
    @constraint(modele, O1_maxSC[m in 1:data.nbMachines, r in 1:data.nbResources],
        maxSC[m,r] >= diffUtiliseesSoft(m,r))
    # ainsi maxSC[m,r] soit nul soit égal à U-SC puisque soit la différence est 
    # négative donc maxSC est nul (puisqu'il doit être positif ou nul) soit 
    # la somme est positive est donc maxSC vaut la différence, on a fait donc max(0,U-SC)
    
    # Il ne reste plus qu'à sommer sur les ressources
    objectif1 = sum(data.weightLoadCost[r]*
        (sum(maxSC[m,r] for m in 1:data.nbMachines)) for r in 1:data.nbResources)
    
    # O2 : Objectif d’équilibrage
    # L'objectif est de répartir équitablement les ressources des machines (ie qu'elles soient toutes utiliées)
    # Mathématiquement on a cout2=somme(sur toutes les machines) max(0,e_{i,j}*A(m,r_i)-A(m,r_j))
    
    # Cette fonction nous permet de faire le calcul e_{i,j}*A(m,r_i)-A(m,r_j)
    function calculDesequilibre(m,b)
        cost_b=data.balanceCostDataList[b]
        return (cost_b.target 
        * (data.hardCapacities[m,cost_b.resource1]
            - ressourcesUtilisees(m,cost_b.resource1))
        - (data.hardCapacities[m,cost_b.resource2]
            - ressourcesUtilisees(m,cost_b.resource2)))
    end
    
    # Tout d'abord nous définissons une variable et nous associons a cette variable une contrainte 
    # Cela nous permet d'effectuer le max de manière analogue à l'objectif précédent 
    @variable(modele, maxEquilibrage[1:data.nbMachines, 1:data.nbBalanceCostData] >= 0, Int)
    @constraint(modele, O2_maxEquilibrage[m in 1:data.nbMachines, b in 1:data.nbBalanceCostData],
        maxEquilibrage[m,b] >= calculDesequilibre(m,b))
    
    # Finalement nous faisons la somme sur la fonction pour obtenir le cout 
    objectif2 = sum(sum(data.balanceCostDataList[b].weight*maxEquilibrage[m,b] 
        for b in 1:data.nbBalanceCostData) for m in 1:data.nbMachines)
 
    # O4 : Objectif de mouvement dans les services
    # Le but est de définir un cout associé au movement dans un service 
    # formellement ce cout va être définit comme 
    # max(pour s dans S)(|{p dans s | M(p) != M_0(p)}|)
    
    # Nous allons proceder de manière analogue à l'objectif 1 et 2 
    
    # Nous allons d'abord créer une fonction qui compte le nombre de mouvement sur un service     
    function nbMouvement(s)
        return sum((data.servicesProcess[p]==s) * (1 - x[p,data.initialAssignment[p]]) 
            # 1 - x[p,m_0] permet de supprimer les process qui n'ont pas bougé
            for p in 1:data.nbProcess)
    end  
    # A ce stade nous avons un fonction permettant de savoir le nombre de mouvement dans un service
    # Il faut donc trouver le max de cette fonction dans S
    # On réutilise le principe de variable et contrainte 
    @variable(modele, deplacementService >= 0, Int)
    @constraint(modele, O4_deplacementService[s in 1:data.nbServices], deplacementService >=
    nbMouvement(s))
    # Cela nous permet d'associer à deplacementService le maximum de nbMouvement(s) pour tous les 
    # services
    
    objectif4 = deplacementService
    
    # O5 : Objectif de mouvement inter-machines
    # Nous voulons ajouter un nouveau cout représentant un cout associé au déplacement d'un processus
    # formellement : cout sur une machine =somme(sur tout les processus de la machine) MMC(M_0(p),M(p))
    objectif5 =  sum(sum(x[p,m]*data.machineMoveCosts[data.initialAssignment[p],m]
                for m in 1:data.nbMachines if m != data.initialAssignment[p]) 
        #le if permet de ne pas comparer la machine avec elle même
                for p in 1:data.nbProcess)


    # Coefficients dans l'objectif 
    # On réunit maintenant les différents couts dans un objectif avec les différents poids
    @objective(modele, Min, objectif1 + objectif2 + data.processMoveWeight 
        * objectif3 + data.serviceMoveWeight * objectif4 + data.machineMoveWeight * objectif5)
    
    ### Fin question 2 ### 
    
    ### Question 3 ### 
    
    # C4 : Contraintes liées aux localisations
    # Nous allons calculer le nombre de localisation d'un service s. 
    # Pour cela nous commençons par créer un tableau analogue à x pour les localisations. 
    # C'est à dire un tableau s_in_l[s,l]
    # Chaque 'case' de ce tableau comporte un 1 si s est dans la localisation l et 0 sinon.
    
    @variable(modele, s_in_l[1:data.nbServices, 1:data.nbLocations] >= 0, Bin)
    
    # On peut maintenant calculer le nombre de localisation de s. 
    # Il s'agit simplement de la somme de tous les elements de la ligne associées à s dans s_in_l.
    function nombreLocalisations(s)
        return sum(s_in_l[s,l] for l=1:data.nbLocations)
    end
    
    # Une fois cette donnée calculée on peut la contraidre sur l'ensemble des services
    @constraint(modele, C4_locations[s=1:data.nbServices], 
        nombreLocalisations(s) >= data.spreadMin[s])
    
    ### Fin question 3 ###
    
    ### Question 4 ###
    
    # C5 : Contraintes liées aux voisinages
    # On cherche maintenant à implémenter une contrainte de voisinages autrement 
    # dit on suppose que certains service nécessitent d'autres services pour s'executer
    # Formellement cela se traduit par ∀ p^a ∈ s^a,∃ p^b ∈ s^b et n ∈ N tel que M(pa) ∈ n et M(pb) ∈ n
    
    # L'idée générale est la suivante 
    # soit p1 dans P et p2 dans P 
    # tel que p1 != p2
    # et tel que service(p2) dépend de service(p1) 
    # alors le voisignage de p2 doit être le même que le voisignage de p1
    
    # Malheuresement après plusieurs échecs nous n'avons pas trouvé une solution
    # La contrainte n'a donc pas été implémentée 
    
    ### Fin question 4 ###
    
    ### Question 5 ### 
    
    # C6 : Le cas des ressources "transient"
    # On veut ajouter une dernière contrainte sur la capacité en prenant en compte des ressources
    # dites transient 
    # Formellement l'idée est la même que C2 cependant maintenant si un processus p est initialement sur 
    # une machine m et passe sur une machine m' alors les ressources "transient" seront consommés sur
    # m et m'
    
    # Maintenant établissons une fonction calculant le cout avec les ressources transient
    function coutTransient(m,r)
            return sum(x[p,m] + (data.initialAssignment[p]==m) * data.processReq[p,r] 
        for p in 1:data.nbProcess)
    end
        
    @constraint(modele, C6_transient[m in 1:data.nbMachines, r in 1:data.nbResources ; 
                data.transientStatus[r] == 1] ,
            coutTransient(m,r) <= data.hardCapacities[m,r])
    
    ### Fin question 5 ###
    
    ### Fin du programme : optimisation et retour des valeurs optimales ###
    optimize!(modele)
    objVal = JuMP.objective_value(modele)
    
    soluce = Array{Int64}(undef, data.nbProcess)
    for p in 1:data.nbProcess #Solution basée sur le post "Projet - erreur d'exécution" du forum
        for m in 1:data.nbMachines
            if JuMP.value(x[p,m])>0.999
                soluce[p]=m
            end
        end
    end
    
    return SolutionGoogle(soluce,objVal)
end


solveGoogle (generic function with 1 method)

### Cellule de test
La cellule ci-dessous permet d'exécuter le code. Elle lit la donnée, lance votre méthode solveGoogle, récupère la solution, et vérifie le respect des contraintes et son coût. 

Vous pouvez modifier le nom pour tester sur un autre jeu de donnée. 

In [40]:
# enter your instance name here 
donnee = "a1_5"  # possibles values a1_1, a1_2, a1_3, a1_4, a1_5, a2_1, a2_2, a2_3, a2_4, a2_5 

# set to yes if you want to print intermediate logs
verbose = false

main("model_"*donnee*".txt","assignment_"*donnee*".txt",verbose)


Received instance files model_a1_5.txt and assignment_a1_5.txt
Checking the solution 
Solution received is [12, 3, 9, 8, 9, 4, 4, 4, 1, 1, 2, 6, 6, 5, 12, 5, 7, 6, 2, 5, 4, 9, 8, 9, 3, 8, 6, 3, 7, 7, 5, 1, 12, 12, 3, 6, 7, 8, 1, 8, 8, 1, 2, 1, 6, 9, 11, 12, 11, 10, 1, 3, 4, 10, 1, 7, 1, 2, 5, 2, 9, 12, 12, 10, 9, 4, 3, 8, 9, 1, 12, 3, 8, 8, 11, 2, 3, 7, 3, 11, 1, 6, 8, 6, 1, 9, 12, 11, 6, 8, 12, 7, 8, 3, 4, 4, 1, 7, 3, 1, 2, 2, 1, 1, 9, 4, 10, 4, 1, 7, 3, 6, 12, 5, 7, 4, 8, 12, 1, 4, 12, 9, 9, 4, 9, 5, 2, 1, 7, 4, 2, 3, 5, 4, 10, 4, 10, 12, 8, 10, 5, 10, 5, 4, 4, 7, 12, 4, 7, 3, 12, 12, 8, 7, 4, 5, 5, 8, 1, 8, 3, 4, 11, 10, 11, 4, 12, 7, 2, 10, 10, 4, 7, 10, 8, 12, 11, 1, 12, 9, 12, 5, 3, 4, 4, 12, 9, 11, 3, 4, 8, 7, 10, 8, 3, 9, 2, 8, 3, 1, 8, 7, 3, 11, 4, 10, 6, 7, 2, 1, 8, 1, 9, 4, 4, 3, 1, 3, 7, 11, 4, 7, 12, 10, 7, 2, 7, 12, 7, 2, 9, 1, 10, 2, 6, 4, 4, 10, 3, 10, 9, 4, 3, 8, 1, 2, 6, 5, 7, 4, 6, 9, 9, 4, 2, 1, 12, 12, 5, 5, 2, 4, 1, 7, 5, 4, 6, 6, 2, 2, 10, 1, 6, 7, 9, 3, 8, 6, 12

## 